In [0]:
!pip install tensorflow-gpu==2.0.0

     |████████████████████████████████| 380.8MB 28kB/s 
     |████████████████████████████████| 3.8MB 33.2MB/s 
     |████████████████████████████████| 450kB 40.4MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=47b5f6941d0c6a22c54e1fa9b618695bb5ae8d32deffb1c541e478743d81f160
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 1.14.0 has requirement tensorboard<1.15.0,>=1.14.0, but you'll have tensorboard 2.0.0 which is incompatible.
ERROR: tensorflow 1.14.0 has requirement tensorflow-estimator<1.15.0rc0,>=1.14.0rc0, but you'll have tensorflow-estimator 2.0.0 which is incompatible.
  Found existing installation: tensorboard 1.14.0
    Uninstalling tensorboard-1.14.0:
      Successfully uninstalled tensorboard-1.14.0
  Found existing installation: tensorflow-estimator 1.14.0
    Uninstalling tensorflow-estimator-1.14.0:
      Successfully uninstalled tens

In [0]:
import tensorflow as tf
tf.__version__

'2.0.0'

Observamos que estamos operando con la versión 2.0.0 de tensorflow, liberada el 30 de septiembre de 2019.

Veamos si tenemos los conocimientos matemáticos necesarios para hacer state-of-the-art deep learning. Esta lección se centrará en lo estrictamente necesario para hacer deep learning.

Definamos de manera sencilla lo que es un grafo. Las redes en deep learning no son más que larguísimas funciones que ante la presencia de decenas de capas se pueden volver terriblemente tediosas de representar. Los grafos nos permiten organizar nuestro pensamiento acerca de las funciones que estamos construyendo, y de hecho mucha investigación actual usa la representación gráfica para desarrollar nuevas y mejores estructuras como lo son las resnet y las efficientnets (más adelante las estudiaremos). Siendo más precisios, las redes neuronales son un tipo particular de grafos, son grafos computacionales. En otras palabras, es más sencillo representar las arquitecturas de nuestras redes por medio de grafos que de por medio de notación matemática; funciones insertadas en funciones insertadas en funciones...

Los grafos computacionales pueden lucir como cualquiera de las siguientes dos arquitecturas, la primera representa la función $(x+y)*z$ y la segunda una arquitectura resnet de 152 capas.

Ahora, para entender las matemáticas necesarias entendamos primero a lo que nos referimos con una red neuronal y en dónde se encuentra cada función que definiremos más adelante.

*Nota: a las redes neuronales también se les llama "artificial neural networks" (ANN) y "perceptrones de múltiples capas".

La siguiente imagen (tomada de la clase CS231n Convolutional Neural Networks for Visual Recognition de Stanford) muestra dos arquitecturas diferentes de red neuronales con capas fully connected (completamente conectadas, en español). En el caso de la izquierda tenemos una red neuronal que cuenta con datos que tienen tres paramétros de entrada. Si nuestros ejemplos son casas, entonces los parámetros pueden ser el número de metros cuadrados que ocupan, la localización y si tienen o no patio). A la primera capa se le llama input layer (capa de entrada, en español). El número de parámetros de la segunda capa es definido por nosotros y a cada parámetro le llamamos unidad computacional o neurona, como se prefiera. En este caso, la arquitectura de la izquierda cuenta con cuatro neuronas. A esta la capa le llamamos hidden layer (capa oculta, español). Finalmente, la output layer (capa de salida) cuenta en este caso con dos neuronas y se utiliza para representar el valor que se le da a cada clase en que podemos clasificar (en el caso de la clasificación de imágenes te indica, por ejemplo, si tu imagen es más probable que sea un gato o un perro; una neurona te da el puntaje con respecto a la clase "perro" y la otra con respecto a la clase "gato".

La arquitectura de la derecha es similar en cuanto que cuenta con capas fully connected pero difiere en que cuenta con dos hidden layer (podemos poner las que queramos, siempre y cuando tomemos en cuenta otros factores con los que lidiaremos más adelante) y con una única neurona en el output layer (podría ser el valor predicho del precio de una acción).

Llamamos a la primera arquitectura como una red neuronal de dos layers y a la segunda como red neuronal de tres layers. Las hidden y output layers definen el número de layers en el nombre de la red.

Dentro de cada neurona suceden, al menos, dos operaciones. Primero, se computa la affine function (función afín, en español): poducto punto entre un vector de pesos y un vector $\vec{x}$. La affine function no es más que una función linear y depende del tipo de red neuronal que queramos construir (por ejemplo, la affine function puede ser una operación de convolución para clasificación de imágenes). Al producto de la affine function sigue una función de no linearidad que suele llevar a los valores negativos a cero. Más adelante entraremos en detalles. Al terminar ambas operaciones decimos que "activamos" la neurona. 

Al considerar todas las neuronas en las hidden layers estamos multiplicando una matriz por otra matriz (tensores de rango dos) 

En esta sesión clasificamos la famosa base de datos MNIST para hacer reconocimiento de números en imágenes. 

In [0]:
(x_train, y_train), (x_val, y_val) = tf.keras.datasets.mnist.load_data()

Por el momento no hagamos caso a nuestros sets de evaluación. En nuestro set de entrenamiento tenemos 6,000 imágenes de 28 x 28 pixeles. Las imágenes están en blanco y negro por lo que no tienen una dimensión RGB. Cada una de las 6,000 imágenes representa un número entre 0 y 9, 10 número en total, y dicha información se encuentra en `y_train`, de forma 60,000 x 1. Podemos dar `x_train.shape, y_train.shape` para conocer esta información. Notar que los datos están en formato numpy, sin embargo al hacer operaciones de tensorflow 2.0 sobre ellos se convierten automáticamente en tensores.

Aplanamos y convertimos a float nuestras imágenes para que podamos operar sobre ellas.

In [0]:
t = tf.reshape(x_train, (60000,-1))
t = tf.dtypes.cast(t, dtype = tf.float32)

Ahora nuestro tensor `t` tiene la forma `TensorShape([6000, 784])` donde el 784 viene de multiplicar 28 x 28.

Guardemos algunas características relevantes de nuestra imagen. `n` será el número de imágenes que tenemos y `m` nuestro número de parámetros iniciales. Además, nh será el número de neuronas que tendremos en nuestra primera capa, ya hablaremos más tarde de eso.

In [0]:
n, m = t.shape
nh1 = 100

`n` es igual a 60,000, el número de imágenes; m es el número de parámetros de nuestro modelo, 784, un valor por cada pixel.

Introduzcamos una estructura de datos que multiplicará a nuestras imágenes y la llamaremos weights (pesos , en español). Definamos las dimensiones que debe tener nuestro tensor de pesos: 



1.   Nuestros weights harán una multiplación de matrices con nuestras imágenes.
2.   Queremos que tenga en la otra dimensión el número de neuronas que definimos para nuestra primera capa, `nh1`.

Nuestros weights entonces tendrán la forma (m, nh). 



In [0]:
w1 = tf.random.normal((m, nh1))*0.0001 

Finalmente, queremos que nuestros pesos tengan una característica muy particular. En cualquier función existen constantes y variables. En deep learning se suele tratar a nuestros datos (imágenes en este caso) como constantes; el valor de nuestros pixeles no cambiará en ningún momento. Nuestras variables serán nuestros weights; los queremos de forma que sean lo más certeros posibles a la hora de predecir lo que sea que queremos predecir. En otras palabras, nosotros operaremos sobre nuestros pesos, los variaremos hasta que encontremos la combinación de pesos que mejor hagan mapping entre nuestros datos y lo que queremos predecir/clasificar. Específicamente, queremos saber como cambia nuestra función de pérdida cada vez que variemos nuestros pesos.

El método `tf.GradientTape()` nos permite registrar los gradientes de una función con respecto a las variables que la integran. En otras, palabras, cuando activamos `tf.GradientTape()` con respecto a una función estamos indicando a tensorflow que eventualmente vamos a requerir los gradientes de dicha función. 

Quedará más claro conforme vayamos construyendo nuestra red neuronal.

*Nota: el recurso de computar los gradientes es más conocido en la librería de pytorch y su método `autograd`. Para los que prefieran una explicación más técnica: el autograd es un "[...] motor para calcular derivadas (producto jacobiano-vector para ser más precisos). Registra un grafo de todas las operaciones realizadas en un tensor con gradiente habilitado y crea un gráfico acíclico llamado gráfico computacional dinámico. Las hojas de este grafo son tensores de entrada y las raíces son tensores de salida. Los gradientes se calculan trazando el grafo desde la raíz hasta la hoja y multiplicando cada gradiente en el camino usando la regla de la cadena." - [Vaibhav Kumar](https://towardsdatascience.com/pytorch-autograd-understanding-the-heart-of-pytorchs-magic-2686cd94ec95).

Para entender mejor el autograd hagamos lo siguiente. Considera la función $$y(x) = x^2,$$ de nuestra clase de cálculo 1 sabemos que $$y'(x) = 2*x,$$ entonces si $x = 5$, $$y'(5) = 2 * 5 = 10.$$ 

Veamos como autograd nos puede dar el mismo resultado. 

In [0]:
x = tf.Variable(5.)

Definimos nuestra función $y(x) = x^2$ y activamos el `tf.GradientTape()` para requerir los gradientes, en este caso la derivada con respecto a x. 

In [0]:
with tf.GradientTape() as tape:
  y = x**2

Ahora usemos la propiedad `gradient()` de nuestro tensor GradientTape, `tape`, para obtener el gradiente de `y` con respecto a `x` y así para conocer el valor de la derivada. *Nota: usemos numpy para imprimir nuestro resultado.

In [0]:
grad = tape.gradient(y,x)
grad.numpy()

10.0

Nos da como resultado 10. Justo lo que buscabamos. Este mismo procedimiento es el que seguiremos más adelante pero con varias funciones compuestas. Como vemos, nada complicado.

Desarrollemos ahora nuestro "array programming". En la programación científica tenemos una preocupación especial por llevar a acabo operaciones en un gran número de valores de manera paralela. El array programming se ve de manera natural en lenguajes como APL, R y Matlab, y en frameworks para Python como numpy o pytorch. Sin array programming es imposible resolver los problemas computacionales que el deep learning demanda de nosotros, punto.

Dicho esto, ahora dictemos que nuestro tensor `x` no sea de rango cero (escalar) sino de rango uno (vector). Como con el caso anterior, queremos que la misma función $x^2$ se aplique para llegar al resultado de `y` y poesteriormente obtener el gradiente con respecto a nuestros valores en `x`. 

In [0]:
x = tf.Variable([5.,4.,3.,2.,1.,0.])
with tf.GradientTape() as tape:
  y = x**2
tape.gradient(y,x).numpy()

array([10.,  8.,  6.,  4.,  2.,  0.], dtype=float32)

Ahora que estamos más familiarizados con el array programming, introduzcamos dos funciones clave para el deep learning: la función de pérdida y la función ReLU. Ninguna representa mayor dificultad.

Existen muchos tipos diferentes tanto de funciones de pérdida como de funciones de no-linearidad. Aquí utilizamos como pérdida la función de "log softmax": 
$$\operatorname{softmax(x)}_{i} = \frac{e^{x_{i}}}{\sum_{0 \leq j \leq n-1} e^{x_{j}}},$$ o en otra notación:
$$\operatorname{softmax(x)}_{i} = \frac{e^{x_{i}}}{e^{x_{0}} + e^{x_{1}} + \cdots + e^{x_{n-1}}}$$

<!-- donde llamaremos a $Y_i$ como "output" (producto, en español) pues será el producto final de nuestra serie de funciones que representan nuestra red neuronal; y a $\hat{Y_i}$ le llamaremos "target" (objetivo, en español) pues es el valor al que aspiramos que nuestro output sea igual. -->
En la práctia aplicaremos logaritmo a nuestra softmax. No se preocupen por la notación matemática, como dice Jeremy Howard, está forma de escribir es bastante de flojera. Ya veremos como la función requiere una única línea de código gracias al array programming. 

Como función de no linearidad utilizaremos la Rectified Linear Unit (ReLU): 
$$\operatorname{ReLU(x)} = \begin{cases}
    x & \text{si } x > 0, \\
    0 & \text{de otra forma}.
\end{cases}$$

Para ambas funciones $x$ será comunmente un tensor de rango dos y gracias al array programming nuestras funciones podrán ser aplicadas simultáneamente a cada elemento dentro de nuestro tensor.

In [0]:
def log_softmax(x): 
  return x - tf.math.log(tf.math.reduce_sum((tf.math.exp(x)), axis=-1, keepdims=True))

Tensorflow tiene su propia implementación de la función de log softmax, `tf.nn.log_softmax`, y podemos comparar su resultado con el de nuestra `log_softmax` y veremos que el resultado es el mismo.

In [0]:
def relu(x):
  return tf.math.maximum(x, 0.)

De la misma manera, tensorflow cueta con la función `tf.nn.relu` y el resultado es el mismo al de nuestra función `relu`.

Estamos listos para multiplicar nuestros pesos por nuestras imágenes.

In [0]:
t_2 = tf.linalg.matmul(t, w1)

In [0]:
t_2.shape

TensorShape([60000, 100])

Ahora tenemos un tensor con dimensiones `TensorShape([60000, 100])`. Es decir, tenemos un tensor rango 1 por cada una de nuestras 60,000 imágenes y cada imagen ahora tiene propiedades en cada una de nuestras 100 neuronas definidas para nuestra capa 1.

Ahora apliquemos nuestra función `relu` a `t_2`.

In [0]:
t_2_relu = relu(t_2)

Notar que al aplicar una función no estamos alterando la forma de nuestros tensores pues la misma función se aplica a cada elemento del tensor. La forma sigue siendo `TensorShape([60000, 100])`.

In [0]:
t_2_softmax = log_softmax(t_2_relu)

Ahora vemos que una red neuronal no es más que una función integrada por múltiples funciones; funciones compuestas. ¿Dónde está el proceso de entrenamiento? Este proceso se hace gracias al algoritmo llamado "backpropagation". Nos queda para la parte 2.